# Chapter 4. Searching and Reading Local Files
### Crawling and searching directories (p. 114)
Download [dir](https://github.com/PacktPublishing/Python-Automation-Cookbook-Second-Edition/tree/master/Chapter04/documents/dir) using DownGit.   [os.path doc](https://docs.python.org/3/library/os.path.html) [OO high-level os.pathlib doc](https://docs.python.org/3/librarpathlib.html)

In [12]:
import os, re
for root, dirs,files in os.walk('dir'):
    print(f'     root={root}')
    for file in files:
        if file.endswith('.pdf'):
            print('PDF===>',end=' ')
        if re.search(r'[13579]', file):
            print('ODD===>',end=' ')
        full_path = os.path.join(root,file)
        print(file, full_path)

     root=dir
file2.txt dir/file2.txt
ODD===> file1.txt dir/file1.txt
PDF===> file6.pdf dir/file6.pdf
     root=dir/subdir
ODD===> file3.txt dir/subdir/file3.txt
file4.txt dir/subdir/file4.txt
PDF===> ODD===> file5.pdf dir/subdir/file5.pdf


### Reading text files (p. 117)
downloaded [zen_of_python.txt](https://github.com/PacktPublishing/Python-Automation-Cookbook-Second-Edition/blob/master/Chapter04/documents/zen_of_python.txt)

In [20]:
with open('zen_of_python.txt') as file:
    for line in file:
        if 'should' in line.lower():
            print(line) 
        if 'dutch' in line.lower():
            print(line)
            break

Errors should never pass silently.

There should be one-- and preferably only one --obvious way to do it.

Although that way may not be obvious at first unless you're Dutch.



### Dealing with encodings (p. 120)
downloaded documents dir. [Encoding aliases](https://docs.python.org/3/library/codecs.html#standard-encodings)

In [ ]:
with open('documents/example_utf8.txt') as file:
    print(file.read())
with open('documents/example_iso.txt', encoding='iso-8859-1') as file:
    print(file.read())

In [ ]:
with open('documents/example_utf8.txt') as file:
    content = file.read()
with open('documents/junk_output_iso.txt', 'w', encoding='iso-8859-1') as file:
    file.write(content)
with open('documents/junk_output_iso.txt', encoding='iso-8859-1') as file:
    print(file.read())

In [32]:
from bs4 import UnicodeDammit
with open('documents/example_iso.txt', 'rb') as file:
    content = file.read()
suggestion = UnicodeDammit(content)
suggestion.original_encoding
suggestion.unicode_markup

'20£'

### Reading CSV files
[csv docs](https://docs.python.org/3/library/csv.html)

In [ ]:
import csv
with open('documents/top_films.csv') as file:
    data = csv.reader('file')
    for row in file:
        print(row)

In [66]:
#with open('documents/top_films.csv') as file:
file =open('documents/top_films.csv')
#  treats first line as headers with KEYs
data = csv.DictReader(file)
structured_data = [row for row in data]
structured_data[0]
structured_data[0].keys()
structured_data[0]['Rank']
file.close()

['excel', 'excel-tab', 'unix']

In [ ]:
csv.list_dialects()
with open('documents/top_films.csv', newline='') as file:
    dialect = csv.Sniffer().sniff(file.read())
    
with open('documents/top_films.csv', newline='') as file:
    reader = csv.reader(file, dialect)
    for row in reader:
        print(row)

### Reading log files
From Chapter 1:

In [80]:
import parse, delorean
from decimal import Decimal

class PriceLog(object):
    def __init__(self,timestamp,product_id,price):
        self.timestamp = timestamp
        self.product_id = product_id
        self.price = price
    def __repr__(self):
        return '<PriceLog({}, {}, {})>'.format(self.timestamp,self.product_id,self.price)
        
    @classmethod
    def parse(cls,log):
        '''
        Parse from a text log with the format
            [<Timestamp>] - SALE - PRODUCT: <product id> - PRICE: $<price>
        to a PriceLog object
        '''
        def price(string):
            return Decimal(string)
        def isodate(string):
            return delorean.parse(string)
        FORMAT = ('[{timestamp:isodate}] - SALE - PRODUCT: {product:d} - PRICE: ${price:price}')
        formats_extra = {'price' : price, 'isodate' : isodate}
        result = parse.parse(FORMAT,log,formats_extra)
        print('result>>>{}<<<<',result)  # yuri's debug output
        return cls(timestamp=result['timestamp'], product_id=result['product'], price=result['price'])

In [85]:
with open('documents/example_logs.log') as file:
    logs = [PriceLog.parse(log) for log in file]
len(logs)
logs[0]
total = sum(log.price for log in logs) #tot sales
total

result>>>{}<<<< <Result () {'timestamp': Delorean(datetime=datetime.datetime(2018, 6, 17, 22, 11, 50, 268396), timezone='UTC'), 'product': 1489, 'price': Decimal('9.99')}>
result>>>{}<<<< <Result () {'timestamp': Delorean(datetime=datetime.datetime(2018, 6, 17, 22, 11, 50, 268442), timezone='UTC'), 'product': 4508, 'price': Decimal('5.30')}>
result>>>{}<<<< <Result () {'timestamp': Delorean(datetime=datetime.datetime(2018, 6, 17, 22, 11, 50, 268454), timezone='UTC'), 'product': 8597, 'price': Decimal('15.49')}>
result>>>{}<<<< <Result () {'timestamp': Delorean(datetime=datetime.datetime(2018, 6, 17, 22, 11, 50, 268461), timezone='UTC'), 'product': 3086, 'price': Decimal('7.05')}>
result>>>{}<<<< <Result () {'timestamp': Delorean(datetime=datetime.datetime(2018, 6, 17, 22, 11, 50, 268468), timezone='UTC'), 'product': 1489, 'price': Decimal('9.99')}>


Decimal('47.82')

how many units have been sold of each product_id.  [cound list doc Counter](https://docs.python.org/3/library/collections.html#counter-objects)

In [86]:
from collections import Counter
counter = Counter(log.product_id for log in logs)
counter

Counter({1489: 2, 4508: 1, 8597: 1, 3086: 1})

### Reading file metadata
meta -- not data content itself -- `zen_of_python.txt`  [doc](https://docs.python.org/3/library/os.html)

In [90]:
import os 
from datetime import datetime
stats = os.stat('zen_of_python.txt')
stats
stats.st_size
datetime.fromtimestamp(stats.st_mtime) #modified
datetime.fromtimestamp(stats.st_atime) #accessed

datetime.datetime(2022, 7, 27, 20, 30, 15, 776877)

In [92]:
#also
os.path.getsize('zen_of_python.txt')
os.path.getmtime('zen_of_python.txt') #unix fmt
os.path.getatime('zen_of_python.txt')

1658979014.5832143

### Reading images
downloaded dir `images`, requirements pillow (Python Image Lib --changed to 9.0.0 for Python 3.10) and xmltodict. 

The metadata information in photo files is defined in the EXIF (Exchangeable Image File) format. __[EXIF](https://www.slrphotographyguide.com/what-is-exif-metadata/)__ is a standard for storing information about pictures, including things like what camera took the picture, when it was taken, GPS describing the location, exposure, focal length, color info.

JPG files store the EXIF info directly, PNG files store __[XMP](https://www.adobe.com/devnet/xmp.html)__ info, a more generic standard that can contain EXIF data inside

Also for GPS cf. ch04-gps_conversion.py

In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import xmltodict
from ch04_gps_conversion import exif_to_decimal, rdf_to_decimal

image1 = Image.open('images/photo-dublin-a1.jpg')
image1.height
image1.width
image1.format

exif_info_1 = { TAGS.get(tag,tag) : value    for tag,value in image1._getexif().items()}  # _getexif(), not getexif()ß
exif_info_1


In [113]:
image2 = Image.open('images/photo-dublin-a2.png')
image2.format
xmp_info = xmltodict.parse(image2.info['XML:com.adobe.xmp'])

rdf_info_2 = xmp_info['x:xmpmeta']['rdf:RDF']['rdf:Description']
rdf_info_2['tiff:Model']
rdf_info_2['exifEX:LensModel']
rdf_info_2['xmp:CreateDate']

1710

In [127]:
exif_info_1['GPSInfo']
gps_info_1 = { GPSTAGS.get(tag,tag) : value   for tag, value in exif_info_1['GPSInfo'].items() }
# exif_to_decimal(gps_info_1) -- TypeError?
rdf_to_decimal(rdf_info_2)

('N53.346905', 'W6.247796666666667')

In [126]:
image3 = Image.open('images/photo-dublin-b.png')
xmp_info = xmltodict.parse(image3.info['XML:com.adobe.xmp'])
rdf_info_3 = xmp_info['x:xmpmeta']['rdf:RDF']['rdf:Description']
rdf_info_3['xmp:CreateDate']
rdf_to_decimal(rdf_info_3)

('N53.34984166666667', 'W6.260388333333333')

For OCR, need to install __[Tesseract](https://github.com/tesseract-ocr/tesseract)__ and __[pytesseract](https://github.com/madmaze/pytesseract)__

Then

In [130]:
import pytesseract
image_text = Image.open('images/photo-text.jpg')
# pytesseract.image_to_string(image_text) -- tesseract not installedß


OpenCV more powerful than Pillow: __[ABANDONDED openCV Python tutorial](http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html)__

### Reading PDF files
[Lorem Ipsum](https://loremipsum.io), second has password `automate`


In [ ]:
from PyPDF2 import PdfFileReader
file = open('documents/document-1.pdf', 'rb')
doc = PdfFileReader(file)
doc.numPages
doc.isEncrypted
doc.documentInfo
doc.pages[0].extractText()
doc.pages[1].extractText()

In [148]:
file.close()
file = open('documents/document-2.pdf', 'rb')
doc = PdfFileReader(file)
doc.isEncrypted
#doc.numPages  #raise error
doc.decrypt('automate')
doc.numPages
doc.pages[0].extractText()
file.close()

More sofcticated PDF readers 
* [pdftooppm](https://linux.die.net/man/1/pdftoppm)
* [QPDF](http://qpdf.sourceforge.net/files/qpdf-manual.html) with Python bindings [pikepdf](https://pikepdf.readthedocs.io/en/stable/). 

### Reading Word documents
[python-docx](https://python-docx.readthedocs.io/en/latest/)

In [ ]:
import docx
doc = docx.Document('documents/document-1.docx')
doc.core_properties.title
doc.core_properties.keywords
doc.core_properties.modified
len(doc.paragraphs)

Walk through the paragraphs to detect the ones that contain text

In [ ]:
for index, paragraph in enumerate(doc.paragraphs):
    if not paragraph.text:
        continue
    print(index,paragraph.text)

In [10]:
doc.paragraphs[30].text

'A VERY IMPORTANT DOCUMENT'

Each of the paragraphs has runs, which are sections of the text with different properties

In [14]:
doc.paragraphs[30].runs[0].italic
doc.paragraphs[30].runs[0].bold
[ run.text for run in doc.paragraphs[48].runs]

['This is an example of a test document that is stored in ',
 'Word',
 ' format',
 '. It contains some ',
 'sentences',
 ' to describe what it is and it has ',
 'lore',
 'm',
 ' ipsum',
 ' text.']

### Scanning documents for a keyword

`ch04-scan.py` searches a word in files

In [177]:
!python ch04-scan.py -w lorem

>>> "lorem" found in ./dir/file6.pdf
>>> "lorem" found in ./dir/subdir/file5.pdf
>>> "lorem" found in ./documents/document-1.pdf
>>> "lorem" found in ./documents/document-1.docx
